Auteur : Michaël Leroy

Notebook reprenant tabular_datas_map_v3.ipynb avec implementation rapide essai isochrones

API 
    * mapbox (je laisse mon token dans le code mais la creation de comptes est gratuite)


A faire

    - voir aux layers
    - voir warning slice dataframe

In [ ]:
# from tqdm.notebook import tqdm
from tqdm import tqdm

import os
os.environ['USE_PYGEOS'] = '0'

# Data management
import pandas as pd
import geopandas as gpd
import numpy as np


# Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.features import Choropleth
from folium.plugins import MarkerCluster

# Preprocessing
from sklearn.preprocessing import StandardScaler
import umap


# I/O
import gc
import io, requests
import zipfile, shutil
import joblib

# tqdm().pandas()

# data_path = 'C:/Users/demo/Desktop/Lattitude/datas/'
data_path = 'datas'
os.makedirs(data_path, exist_ok=True)

------------------------------------------------

# Datas

In [ ]:
# Load the data for French garge points
file_name = 'dataset_charge_points.feather'

bornes = gpd.read_feather(os.path.join(data_path, file_name))

# Load the data for the communes
file_name = 'dataset_communes.feather'

communes = gpd.read_feather(os.path.join(data_path, file_name))

In [ ]:
bornes.crs 

In [ ]:
bornes.plot()

-------------------------------------------------------

# Create some metric on electrics stuff

In [ ]:
communes['VE_pct'] = communes.nb_vp_rechargeables_el / communes.nb_vp
communes['VE_per_inhab'] = communes.nb_vp_rechargeables_el / communes.PMUN
communes['VE_per_ha'] = communes.nb_vp_rechargeables_el / communes.surf_ha


----------------------------

# Select a town

In [ ]:
TOWN = 'Reims'

In [ ]:
def single_popup(df):
    texts = []

    # loop on rows
    for index, row in df.iterrows():
        text = ''
        for col in communes_info_cols:
            text += f'<b>{col}:</b> {row[col]}<br>'
        texts.append(text)
    return texts    

# list of columns from datas_ to be displayed
communes_info_cols = communes.columns.tolist()
communes_info_cols.remove('geometry')
communes_info_cols.remove('wikipedia')


# communes['html_popup'] = popup(communes)   
# communes.html_popup.head(2)

In [ ]:
# Column names generator 
def split_list(list_a, chunk_size):
  for i in range(0, len(list_a), chunk_size):
    yield list_a[i:i + chunk_size]


def column_popup(df, info_cols,num_cols=3,width=20):
    # Mise en page
    num_cols = 3
    width = 20
    px = np.ceil(width / num_cols / 2)

    texts = []

    # Loop on rows
    for index, row in df.iterrows():
        text = f'<table style="width:{width}%"><tr>'
        # Create header
        for n in range(num_cols):
          text += f'<td style="font-weight:bold">{n}</td>'
        # lines
        for cols in split_list(bornes_info_cols,num_cols):
                text += '<tr>'
                for col in cols:
                        text += f'<th style="width:{px}%"><b>{col}:</b><br> {row[col]}</th>' 
                text  += '</tr>'
        # ends table         
        text += '</table>'
        # append to previous
        texts.append(text)
    
    return texts

 # list of columns from datas_ to be displayed
bornes_info_cols = bornes.columns.tolist()
bornes_info_cols.remove('geometry')   


# bornes_1['html_popup'] = column_popup(bornes_1, info_cols=bornes_info_cols)   
# bornes_1.html_popup.head(2)

------------------------------------------------

# Map

In [ ]:
from branca.colormap import linear, LinearColormap

# Define the color map
colors = [ 'red', 'green', 'blue']
bins = np.array([0, 1, 5, 10, 25, 50, 100]) / 100

labels = [1,2,3,4,5,6]

cmap = LinearColormap(colors=colors, vmin=1, vmax=6)

'''color argument of Icon should be one of: 
{'red', 'darkred', 'gray', 'blue', 'black', 'darkpurple', 'white', 'darkblue', 
 'purple', 'lightred', 'green', 'orange', 'cadetblue', 'beige', 'lightblue', 
 'lightgray', 'darkgreen', 'pink', 'lightgreen'}.'''
icon_labels = ['darkred','lightred','lightgreen','darkgreen','lightblue','blue']

# Function to make a column color 
def make_color(df, col='VE_per_inhab', color_type=None):
    if color_type:
        color =pd.cut( df[col], #.apply(lambda x:  x ** (1/3)),
                        bins=bins, 
                        labels=icon_labels)
    else:
        color =pd.cut( df[col], #.apply(lambda x:  x ** (1/3)),
                            bins=bins, 
                            labels=labels).apply(cmap)
    return color

def make_color_by_date(df, date_col='date_arrete', col='VE_per_inhab', color_type=None, icons=None):
    if icons:
        df['color'] = 'white'
    else:
        df['color'] = '#000000'

        for date in df[date_col].unique():
            df.loc[df[date_col] == date,'color'] = make_color(df.loc[df[date_col] == date], col, icons)
    
    return df


# display(make_color_by_date(communes, 'VE_per_inhab').color.value_counts())  # trés long sur toute la base

# display(make_color_by_date(communes, 'VE_per_inhab', icons='ok').color.value_counts())    




In [ ]:
'''
icons from https://fontawesome.com/v4/icons/

'''
from folium.plugins import MarkerCluster
from folium import FeatureGroup
import math




def make_map(df,com_df, pdc_df, color_col='VE_per_inhab', info_cols=communes_info_cols):
    # Get center
    center = df.iloc[0,1]
    
    # Create a folium map centered on the town_
    m = folium.Map(location=[center.y, center.x], zoom_start=10, crs='EPSG3857')

    

    # Create colors by binning the VE_per_inhab column
    # com_df = make_color_by_date(com_df, 'VE_per_inhab',icons='XXX')    

    # communes markers
    print('Adding isochrones...', end=' ')
    for index, row in df.iterrows():
        # style the polygons based on "values" property
        # color = cmap(row.VE_per_inhab ** (1/3))
        def style_fn(feature):
            ss = {
                "fillColor": 'blue',
                "fillOpacity": 0.09,
                "weight": 0.3,
                "color": 'black',
            }
            # print(row['style'], end='   ')
            return ss   #row['style']
        if index > 0:    
            group = folium.FeatureGroup(name=row.layer_name)

            folium.GeoJson(row.geometry, style_function=style_fn
                                ).add_to(group)
            group.add_to(m)
        # print('done')   
        if index == 1:           # iso = 1 is the largest polygon
            mask = row.geometry                     
        del style_fn
    print('done.')    

    # Take POI in larger isochrone
    print('Ceating communes popus...', end=' ')
    com_df = com_df.copy()[com_df.within(mask)]
    com_df['html_popup'] = single_popup(com_df)
    com_df = make_color_by_date(com_df, col=color_col)
    print('done.')

    print('Creating bornes popup...', end=' ')
    pdc_df = pdc_df.copy()[pdc_df.within(mask)]
    pdc_df['html_popup'] = column_popup(pdc_df, info_cols=info_cols) 
    print('done.')

    print('Creating communes clusters...', end=' ')

    # create custom cluster icon customization function in JS
    icon_create_function = """\
    function(cluster) {
    return L.divIcon({
    html: '<b>' + cluster.getChildCount() + '</b>',
    className: 'marker-cluster marker-cluster-large',
    iconSize: new L.Point(30, 30)
    })
    }"""

    # Create a marker cluster layer for the data
    cluster_com = MarkerCluster(name='Communes', icon_create_function=icon_create_function )

    # communes markers
    for index, row in com_df.iterrows():
        #communes child markers
        popup = folium.Popup(row.html_popup, parse_html=False)
        cluster_com.add_child(folium.Marker(location=[
                                                    row.geometry.centroid.y,
                                                    row.geometry.centroid.x
                                                ], 
                                            popup=popup, 
                                            tooltip=row.date_arrete,   #.strftime('%d/%m/%Y'),
                                            icon=folium.Icon(
                                                        prefix='fa', 
                                                        icon='institution', 
                                                        color=row.color
                                                        )
                                            )
        )
        
        
    cluster_com.add_to(m)
    print('done.')

    print('Creating bornes clusters...', end=' ')
    # Create a marker cluster layer for the data
    cluster_bdr = MarkerCluster(name='Points de charge')

    # bornes markers
    for index, row in pdc_df.iterrows():
        popup = folium.Popup(row.html_popup, parse_html=False)
        cluster_bdr.add_child(folium.Marker(location=[row.geometry.y, row.geometry.x], 
                                            popup=popup, 
                                            tooltip='pdc infos',
                                            icon=folium.Icon(prefix='fa', icon='bolt', color='blue')))
        
    cluster_bdr.add_to(m)
    print('done.')

    # Add a layer control to the map
    folium.LayerControl().add_to(m)

    return m

# m = make_map(centre_df, bornes_1)

# display(m)



In [ ]:
origin = communes.query("nom == @TOWN").iloc[0].geometry.centroid

d = {'name': ['reims'], 
     'geometry': [origin], 
     'style': [
        {
            'fill': 'black',
            'fillOpacity': 1.0,
            'fill-opacity': 0.33,
            'fillColor': 'black',
            'color': 'black',
            'contour': 0,
            'opacity': 1.0,
            'weight': 10,
            'metric': 'distance'
        }
             ],
     'layer_name': ['ville' ]       
}


In [ ]:
token = 'pk.eyJ1IjoibWlrYWxlcm95IiwiYSI6ImNsZzZvcDN4dDBmbXMzZHFmYmd1ajJ6bGIifQ.-GfBZA2ZGXHpkELd0eLBAw'

In [ ]:
iso = [0,0,0,0]

iso[0] = 20e3
iso[1] = 50e3
iso[2] = 75e3
iso[3] = 100e3

denoise = 0   #for polygons, 0 for road trace

request = f'https://api.mapbox.com/isochrone/v1/mapbox/driving/'\
    + f'{origin.x}%2C{origin.y}?'\
    + f'contours_meters={iso[0]}%2C{iso[1]}%2C{iso[2]}%2C{iso[3]}'\
    + f'&polygons=true&denoise=1&access_token={token}'

In [ ]:
isos = requests.get(request).json()   
# isos.keys()

In [ ]:
from shapely.geometry import Polygon, Point

In [ ]:

for i, feature in enumerate(isos['features']):
    # print(i,feature['geometry']['coordinates'][0])
    points = [ Point(point) for point in feature['geometry']['coordinates'][0] ]
    # print(points)
    # print(Polygon(points))
    d['name'].append(f'iso_{i}')
    d['geometry'].append( Polygon( points ))
    style = feature['properties']
    style.pop('fill-opacity')
    style.pop('fill')
    style.pop('contour')
    style.pop('metric')
    d['style'].append(feature['properties'])
    d['layer_name'].append( str(iso[i])[:-5] + ' km')

    # break

In [ ]:
centre_df = gpd.GeoDataFrame(d, crs='EPSG:3857')    #"EPSG:4326")
centre_df

In [ ]:
m = make_map(centre_df, communes, bornes, info_cols=communes_info_cols)

m

Trouver le moyen de tweeker le css des clustermarker une seule coleur par groupe de cluster.

la solution est peu être de modifier le html de la carte après l'avoir générée

In [ ]:
maps_path = 'datas/maps'
os.makedirs(maps_path, exist_ok=True)

file_name = f'isochrones {TOWN}.html'

m.save(os.path.join(maps_path,file_name))

del m